# First Steps

In this tutorial, we'll create our first fully simulated scenario: a server connected to solar power with battery backup.

In [ ]:
import vessim as vs

# For plotting results
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Required for running Mosaic in Jupyter notebooks (fixes asyncio event loop conflicts)
import nest_asyncio
nest_asyncio.apply()

We'll create a single microgrid with:

- **A server** that constantly uses 700W.
- **A solar power system** that produces up to 5kW (using real weather data provided by [Solcast](https://solcast.com).
- **A battery** that can store up to 1500Wh of energy (starts 80% charged, never goes below 30%). We use a `vs.SimpleBattery` which has no (dis)charge limits or losses.
- **A monitor** that writes all progress to a CSV file.

The simulation covers a 24 hour period in Berlin on June, 2022.

In [2]:
# Create the simulation environment
environment = vs.Environment(sim_start="2022-06-09 00:00:00")

# Logs all relevant simulation data to a CSV file
monitor = vs.Monitor(outfile="result.csv")

# Add a single microgrid to the simulation
environment.add_microgrid(
    actors=[
        # Server that consumes 700W constantly
        vs.Actor(
            name="server",
            signal=vs.ConstantSignal(value=-700),  # negative = consumes power
        ),
        # Solar panel that produces up to 5kW based on the Berlin dataset provided by Solcast
        vs.Actor(
            name="solar_panel",
            signal=vs.Trace.load(
                "solcast2022_global", 
                column="Berlin", 
                params={"scale": 5000}  # 5kW maximum
            ),
        ),
    ],
    controllers=[
        monitor
    ],
    storage=vs.SimpleBattery(
        capacity=1500,      # 1500Wh capacity
        initial_soc=0.8,    # Start 80% charged
        min_soc=0.3         # Never go below 30%
    ),
    step_size=300,  # Simulates the microgrid in 5min steps
)

# Run the simulation for 24 hours
environment.run(until=24 * 3600)

2025-06-27 13:19:30.135 | INFO     | mosaik.async_scenario:start:361 - Starting "Actor" as "server" ...
2025-06-27 13:19:30.136 | INFO     | mosaik.async_scenario:start:361 - Starting "Actor" as "solar_panel" ...
2025-06-27 13:19:30.138 | INFO     | mosaik.async_scenario:start:361 - Starting "Grid" as "Grid-0" ...
2025-06-27 13:19:30.139 | INFO     | mosaik.async_scenario:start:361 - Starting "Controller" as "Monitor-0" ...
2025-06-27 13:19:30.140 | INFO     | mosaik.async_scenario:start:361 - Starting "Storage" as "Storage-0" ...
2025-06-27 13:19:30.141 | INFO     | mosaik.async_scenario:run:697 - Starting simulation.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86400/86400 [00:00<00:00, 367187.37steps/s]
2025-06-27 13:19:30.407 | INFO     | mosaik.async_scenario:run:753 - Simulation finished successfully.


## Results

We load the results from disc and take a look at the first few rows. Alternatively, you can access the logs in-memory via `monitor.log`.

In [3]:
df = pd.read_csv("result.csv", parse_dates=[0], index_col=0)
df.head()

,p_delta,p_grid,server.p,solar_panel.p,policy.mode,policy.charge_power,storage.soc,storage.charge_level,storage.capacity,storage.min_soc,storage.c_rate
time,,,,,,,,,,,
2022-06-09 00:00:00,-700.0,0.0,-700,0.0,grid-connected,0.0,0.800000,1200.000000,1500,0.3,NaN
2022-06-09 00:05:00,-700.0,0.0,-700,0.0,grid-connected,0.0,0.761111,1141.666667,1500,0.3,NaN
2022-06-09 00:10:00,-700.0,0.0,-700,0.0,grid-connected,0.0,0.722222,1083.333333,1500,0.3,NaN
2022-06-09 00:15:00,-700.0,0.0,-700,0.0,grid-connected,0.0,0.683333,1025.000000,1500,0.3,NaN
2022-06-09 00:20:00,-700.0,0.0,-700,0.0,grid-connected,0.0,0.644444,966.666667,1500,0.3,NaN


## Visualization

The log contains metrics from all consumers, producers, the battery, and the grid.
Let's visualize the results to see the key patterns:

- **At night**: No solar power, so the battery provides power until it's depleted, then power comes from the public grid.
- **During the day**: Solar panels produce more power than the server needs, so excess energy charges the battery. Once the battery is full, energy is fed back to the public grid.
- **At evening**: Solar power decreases and the battery gets discharged.

Note:
- **Delta power** describes the sum over all actors' power (consumers and producers), so it's the current deficit or excess of electricity.
- **Grid power** describs the power that has been drawn or fed into the public grind during the last simulation step, hence, after considering the charging/discharging of batteries.

In [ ]:
# Create subplots with shared x-axis
fig = make_subplots(
    rows=2, cols=1, 
    shared_xaxes=True,
    row_heights=[0.67, 0.33],
    subplot_titles=("Solar vs Grid Power Over 24 Hours", "Battery State of Charge Over 24 Hours"),
    vertical_spacing=0.1
)

# Top subplot: Solar vs Grid Power
fig.add_trace(
    go.Scatter(x=df.index, y=df["server.p"], name="Server power", line=dict(color="red")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["solar_panel.p"], name="Solar power", line=dict(color="orange")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["p_delta"], name="Delta power", line=dict(color="gray")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["p_grid"], name="Grid power", line=dict(color="blue")),
    row=1, col=1
)

# Bottom subplot: Battery State of Charge
fig.add_trace(
    go.Scatter(
        x=df.index, 
        y=df["storage.soc"] * 100, 
        name="Battery SoC", 
        line=dict(color="green"),
        fill='tozeroy',
        fillcolor='rgba(0,128,0,0.1)'
    ),
    row=2, col=1
)

# Add minimum SoC line
fig.add_hline(
    y=30, 
    line=dict(color='gray', dash='dash', width=1),
    annotation_text="Min SoC (30%)",
    annotation_position="top right",
    row=2, col=1
)

# Update layout
fig.update_layout(
    height=600,
    showlegend=True,
    hovermode='x unified',
    margin=dict(l=0, t=60, b=0, r=0)
)

# Update x-axis labels
fig.update_xaxes(title_text="Time", row=2, col=1)

# Update y-axis labels
fig.update_yaxes(title_text="Power (W)", row=1, col=1)
fig.update_yaxes(title_text="State of Charge (%)", range=[0, 100], row=2, col=1)

# Show the plot
fig.show()